In [1]:
import os
import re
import wandb
import pandas as pd

In [2]:
from constants import WANDB_PROJECT

In [3]:
%%time
api = wandb.Api(timeout=30)
runs = api.runs(WANDB_PROJECT)

pattern = r"^(.*?)\s(.*?)\s(.*)$"
df = pd.DataFrame()
df_cap = pd.DataFrame()

for run in runs:
    if run.state == 'running' or 'N=' in run.name or '4x' in run.name:
        continue
    
    tmp_df = pd.DataFrame(run.scan_history())
    tmp_df['id'] = run.id
    
    match = re.match(pattern, run.name)
    if match:
        dataset = match.group(1)
        transformation = match.group(2)
        method = match.group(3)
        tmp_df['dataset'] = dataset
        tmp_df['transformation'] = transformation
        tmp_df['method'] = method
        
        df = pd.concat([df, tmp_df])

df = df.drop(columns=['_runtime', '_timestamp'])
df = df.drop(columns=['_step'])

CPU times: user 1.79 s, sys: 57.1 ms, total: 1.85 s
Wall time: 4min 9s


In [4]:
# rotation only
dff = df.drop(columns=['Accuracy per Scale Factor', 'Angle', 'Scale Factor', 'train_accuracy',
                       'learning_rate', 'test_accuracy', 'test_loss', 'train_loss', 'id']).dropna()

In [5]:
for ds in ['GTSRB', 'CIFAR10']:
    dfg = dff.loc[dff['dataset'] == ds].groupby(['dataset', 'transformation', 'method']).mean()
    
    da_mean = dfg.query('method == "DA"').mean().mean()
    inverse_mean = dfg.query('method == "Inverse"').mean().mean()
    scn_mean = dfg.query('method == "SCN D=3" or method == "SCN D=5	"').mean().mean()
    
    print(f'{ds:7} Inverse: {inverse_mean - da_mean}, SCN: {scn_mean - da_mean}')

GTSRB   Inverse: 0.06208665497448973, SCN: 0.043378241921768645
CIFAR10 Inverse: 0.014304641812865393, SCN: 0.07565366867690049
